In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/filtered_reviews_in_Phonex.csv')
data.head(3)

# this data is based on the output of 1.0.0_Data_Filtering

,Unnamed: 0,review_id,user_id,business_id,stars,date,business_name,city,state,categories
0,1,aBWKb49Xfkv1946YN7_SIg,sSPbLBHcEMXaJfoO8zs1bA,poSV39UqEg-gpESXafS9-g,5.0,2016-01-17 05:33:14,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
1,2,jCHaWXgppmZjkOdpFltWGA,D5ywfFmwtJxLReqAYlDDmw,poSV39UqEg-gpESXafS9-g,5.0,2016-01-30 01:13:29,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
2,3,KvJ8yT-dODmCYe21J-Pp_A,gjoN4q-f61kwjmmU4mm1_g,poSV39UqEg-gpESXafS9-g,2.0,2016-03-30 22:28:36,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."


In [3]:
'''
construct rating matrix from data
the columns of which represent business_id
the rows of which represent user_id
the values of whose elements represent the according ratings
'''

user_id = data['user_id'].unique().tolist() #rows of sparse matrix
busi_id = data['business_id'].unique().tolist() #columns of sparse matrix

train_sparse_matrix = np.zeros(shape=(len(user_id), len(busi_id)))
test_sparse_matrix = np.zeros(shape=(len(user_id), len(busi_id)))

# construct train_test_split 
ranking_df = data[['user_id','business_id','stars','date']].groupby(['user_id'])

train_user_id = []
train_business_id_lst = []
train_avg_stars = []

test_user_id = []
test_business_id_lst = []
test_new_star = []

for group_name, group_df in ranking_df:
    if len(group_df) > 1:
        new_df = group_df.iloc[-1, :]
        test_user_id.append(new_df['user_id'])
        test_business_id_lst.append(new_df['business_id'])
        test_new_star.append(new_df['stars'])
        
        
        past_df = group_df.iloc[:-1, :]
        past_df['avg_stars'] = past_df.loc[:, 'stars'].mean()
        train_user_id.append(past_df['user_id'].tolist()[0]) 
        train_business_id_lst.append(past_df['business_id'].tolist())
        train_avg_stars.append(past_df['avg_stars'].tolist()[0])

    else:
        past_df = group_df
        past_df['avg_stars'] = past_df.loc[:, 'stars'].mean()
        train_user_id.append(past_df['user_id'].tolist()[0])
        train_business_id_lst.append(past_df['business_id'].tolist())
        train_avg_stars.append(past_df['avg_stars'].tolist()[0])

train_df = pd.DataFrame({'user_id': train_user_id, 'business_id': train_business_id_lst, 'stars': train_avg_stars})
test_df = pd.DataFrame({'user_id': test_user_id, 'business_id': test_business_id_lst, 'stars': test_new_star})


for i in range(len(train_df)):
    element = train_df.iloc[i, 2]
    row_index  = user_id.index(train_df.iloc[i, 0])
    for j in train_df.iloc[i, 1]:
        column_index = busi_id.index(j)
        train_sparse_matrix[row_index, column_index] = element
        
for user, business, stars in zip(test_df['user_id'].tolist(), test_df['business_id'].tolist(), test_df['stars'].tolist()):
        train_sparse_matrix[user_id.index(user), busi_id.index(business)] = 0
        test_sparse_matrix[user_id.index(user), busi_id.index(business)] = stars

for i in range(len(test_df)):
    element = test_df.iloc[i, 2]
    row_index, column_index = user_id.index(test_df.iloc[i, 0]), busi_id.index(test_df.iloc[i, 1])
    test_sparse_matrix[row_index, column_index] = element

# calculate sparstiy of the matrix
train_sparsity = 1 - np.count_nonzero(train_sparse_matrix)/ (train_sparse_matrix.shape[0] * train_sparse_matrix.shape[1])
test_sparsity = 1 - np.count_nonzero(test_sparse_matrix)/ (test_sparse_matrix.shape[0] * test_sparse_matrix.shape[1])
train_sparsity *= 100
test_sparsity *= 100

print (f'{len(user_id)} users')
print (f'{len(busi_id)} business')
print (f'Train Sparsity: {round(train_sparsity,4)}%')
print(f'Test Sparsity:  {round(test_sparsity,4)}%')

# return train_sparse_matrix, test_sparse_matrix

/Users/zhangwenxin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/zhangwenxin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


20472 users
1782 business
Train Sparsity: 99.4425%
Test Sparsity:  99.944%


In [6]:
np.save('train_sparse_matrix.npy', train_sparse_matrix)

In [7]:
np.save('test_sparse_matrix.npy', test_sparse_matrix)